# Предисловие

Всем привет! Решил начать писать конспекты к нашим лекциям в формате ноутбуков. Большинство дисциплин для меня новые, поэтому посчитал, что такие конспекты будет полезно писать, потому что информация очень хорошо откладывается. Буду делать это для дисциплин, где есть код — так получается в быстрее и нагляднее. Плюс, можно сразу всё залить на GitHub и делиться. Буду рад, если кому-то окажется полезным почитать. Можете звёздочку репозиторию кинуть, будет приятно :)

Этот конспект лучше всего изучать параллельно с лекцией — в отрыве от неё эффективность будет ниже. Каждый тоггл соответствует названию слайда. Если какого-то слайда нет отдельным блоком, значит, я раскрыл тему в предыдущих разделах.

Все примеры кода рабочие, иначе специально демонстрируют ошибку. Чтобы их выполнить, залейте этот ноутбук на учебный сервер и подключитесь к учебной базе данных. Инструкция, как это сделать, лежит в официальном ноутбуке ко второму семинару.

Peace \o/

# Язык структурированных запросов. Реляционная модель данных

## Реляционная модель данных 1/5

**База данных** — это совокупность данных, которая содержит информацию в структурированном виде в виде таблиц.

**Каждая таблица имеет  столбцы и строки:**
- cтолбец, или атрибут, — это элемент таблицы, который определяет, **какую** информацию мы храним. Например, `имя`, `фамилия`, `дата рождения`;
- строка, или запись, — это элемент таблицы, который определяет, **что** за информацию мы храним, Например, `Иван`, `Петров`, `14-02-2001`. Каждая строка является экземпляром сущности.

**Экземпляр сущности** — это конкретный объект, который хранится в таблице.

Чтобы таблицы работали как единая система, им нужны правила, которые помогают их организовывать. Эти правила определяет модель данных.

**Модель данных** — это набор правил, которые определяют, как:
- организовать данные в таблицах;
- связать таблицы между собой;
- обеспечить целостность информации.

Одной из самых популярных и эффективных моделей данных является реляционная.

**Реляционная модель данных** — это способ организовывать данные в виде связанных таблиц, или отношений.

## Реляционная модель данных 2/5

В этом и следующих разделах вы встретите примеры кода. Если пока не понимаете, что означает каждый оператор или как работает код целиком — это ок. Ваша задача — просто запустить эти примеры и посмотреть вывод.

**Реляционная модель данных состоит из шести элементов:**

1. **Схема** — это формальное описание структуры базы данных. Она определяет, что находится в таблицах: столбцы, типы данных, первичные и внешние ключи. Например, схема базы cars определяет, что она содержит таблицы `car`, `class`, `driver`, `res, а также какие связи есть между ними.

In [ ]:
%%sql
SELECT 
    'Схема базы ' || current_database() AS описание_схемы,
    COUNT(*) AS количество_таблиц
FROM information_schema.tables 
WHERE table_schema = 'public';

2. **Таблица** — это структура, которая хранит данные в реляционной модели. Она представляет один класс объектов в виде набора строк и столбцов. Например, таблица `car` хранит информацию обо всех автомобилях в базе.

In [ ]:
%%sql
SELECT 
    'сar' AS объект,
    'Хранит данные об автомобилях' AS назначение
FROM information_schema.tables 
WHERE table_name = 'car';

3. **Столбец** — это элемент таблицы, который задаёт свойство объектов, которые мы храним. Он имеет имя и тип данных. Например, в таблице `car` есть столбцы `make` — текст, `year` — целое число, `mileage` — целое число.

In [ ]:
%%sql
SELECT 
    column_name AS имя_столбца,
    data_type AS тип_данных,
    CASE 
        WHEN column_name = 'make' THEN 'Марка автомобиля'
        WHEN column_name = 'year' THEN 'Год выпуска'
        WHEN column_name = 'mileage' THEN 'Пробег в км'
        ELSE 'Другое свойство'
    END AS что_хранит
FROM information_schema.columns 
WHERE table_name = 'car'
ORDER BY ordinal_position;

4. **Строка** — это элемент таблицы, который содержит данные о конкретном объекте. Она представляет собой набор значений для всех столбцов таблицы. Например, в таблице `car` строка может содержать значения `Toyota Camry`, `2020`, `35000`.

In [ ]:
%%sql
SELECT 
    'Строка ' || ROW_NUMBER() OVER (ORDER BY cid) AS номер_строки,
    make AS марка,
    year AS год,
    mileage AS пробег
FROM car 
ORDER BY cid 
LIMIT 3;

5. **Первичный ключ** — это столбец или набор столбцов таблицы, чьи значения определяют каждую её строку. Первичный ключ гарантирует уникальность записей. Например, в таблице `car` столбец `cid` со значениями `101`, `102`, `103` является первичным ключом.

In [ ]:
%%sql
SELECT 
    'cid' AS ключ,
    COUNT(DISTINCT cid) AS уникальных_значений,
    COUNT(*) AS всего_записей,
    CASE 
        WHEN COUNT(DISTINCT cid) = COUNT(*) 
        THEN 'Все значения уникальны' 
        ELSE 'Есть дубликаты' 
    END AS проверка_уникальности
FROM car;

6. **Внешний ключ** — это столбец или набор столбцов таблицы, чьи значения должны совпадать со значениями первичного ключа в другой таблице. Он устанавливает логическую связь между таблицами. Например, в таблице `car` столбец `clsid` содержит значения, которые ссылаются на первичный ключ в таблице `class`.

In [ ]:
%%sql
SELECT 
    'clsid' AS ключ,
    'class' AS ссылка,
    COUNT(DISTINCT clsid) AS уникальных_классов,
    (SELECT COUNT(*) FROM class) AS всего_классов_в_справочнике
FROM car;

Чтобы получить полные данные об автомобиле, мы связываем таблицы через общее поле.

In [ ]:
%%sql
SELECT
    c.cid AS id_автомобиля,
    c.make AS марка,
    c.clsid AS код_класса,
    cl.capacity AS вместимость,
    cl.transmission AS трансмиссия
FROM car c
JOIN class cl ON c.clsid = cl.clsid
LIMIT 5;

Реляционная модель работает так, чтобы данные не повторялись. Например, свойства класса автомобиля — вместимость и тип трансмиссии — находятся в таблице `class`. Таблица `car` не дублирует эти свойства, а ссылается на них через поле `clsid`.

**Избыточность данных** — это явление, при котором в базе данных одна информация повторяется несколько раз.

Реляционная модель позволяет управлять избыточностью данных через ключи. Она не дублирует данные, а заменяет их ссылками. Например, таблица `car` содержит только ключ `clsid`. Сами данные о классе хранят один раз в таблице `class`. Автомобиль связывается с классом, когда значение `clsid` в таблице `car` совпадает со значением `clsid` в таблице `class`.

**Сравним два подхода к хранению данных:**
1. Записывать свойства класса в строку каждого автомобиля. Это создает дубликаты в данных.
2. Хранить свойства класса один раз в отдельной таблице, а в таблице автомобилей оставлять только ссылку.

Этот запрос показывает, как выглядели бы данные, если бы свойства класса хранили напрямую в таблице car. Значения `Capacity: 5` и `Transmission: a` физически повторяются для каждого автомобиля класса `D`. Данные `5` и 'a' дублируются в каждой строке.

In [ ]:
%%sql
SELECT
    cid,
    make,
    clsid,
    'Capacity: 5, Transmission: a' AS class_info_duplicated
FROM car
WHERE clsid = 'D'
LIMIT 3;

На самом деле эти данные хранятся один раз. Узнаем их источник. Этот запрос выводит свойства класса `D` из таблицы `class`. Мощность `5` и трансмиссия `a` записываются один раз. Здесь нет повторений. Это позволяет эффективно хранить данные и не перегружать базу.

In [ ]:
%%sql
SELECT 
    clsid,
    capacity,
    transmission
FROM class 
WHERE clsid = 'D';

Свяжем таблицы, чтобы получить данные без дублирования. Этот запрос объединяет таблицы. `car` содержит только ключ `clsid`. Система находит в `class` строку, где `clsid` совпадает и добавляет оттуда `capacity` и `transmission`. В результате данные о классе не дублируются, так как система использует их из одной записи.

In [ ]:
%%sql
SELECT 
    c.cid,
    c.make,
    c.clsid,
    cl.capacity,
    cl.transmission
FROM car c
JOIN class cl ON c.clsid = cl.clsid
WHERE c.clsid = 'D'
LIMIT 3;

Сравним результат с первым запросом. Здесь те же данные `Capacity: 5` и `Transmission: a`, но они не дублируются в базе. Система берёт их из одной записи таблицы `class` по совпадению `clsid`.

**Введём два новых определения:**
1. **Нормализация** — это процесс, который устраняет дублирование данных и разделяет информацию на таблицы. Этот процесс показан выше на примере `car` и `class`.
2. **Денормализация** — это обратный процесс, который допускает дубли в данных, чтобы запросы выполнялись быстрее.

Например, база `cars` в её текущем виде является нормализованной. Таблицы `car` и `class` разделены, а данные не дублируются.

Инженеры данных и архитекторы применяют нормализацию и денормализацию для разных задач. Например, если необходимо обеспечить целостность данных при постоянных изменениях, для этого проектируют **операционную базу данных**.

**Операционная база данных** — это система, которая обрабатывает постоянные транзакции приложения. Она постоянно записывает, обновляет, удаляет и считывает отдельные записи. 

Например, база данных онлайн-банка. Она обрабатывает тысячи транзакций каждую минуту: списывает и зачисляет деньги, обновляет балансы пользователей и фиксирует новые платежи. Такие данные должны оставаться точными, так как постоянно меняются. Чтобы добиться целостности данных, эту базу **нормализуют**. Её делят на таблицы `Клиенты`, `Счета` и `Транзакции`. Это позволяет гарантировать, что каждая операция не нарушит целостность данных. Например, личные данные клиента изменяются только в одной таблицы `Клиенты`, а все счета, которые с ней связаны, ссылаются на актуальную запись. 

Если же необходимо ускорить работу сложных запросов, для этого проектируют **аналитическую базу данных**.

**Аналитическая база данных** — это система, которая хранит данные для анализа и отчётности. Она редко изменяет данные, но часто используется, чтобы быстро выполнять сложные запросы по большим объёмам информации.

Например, база данных отчётов для топ-менеджмента банка. Она уже не обрабатывает платежи, а позволяет посчитать общую выручку за год, построить графики по регионам, выявить тренды. Чтобы эффективнее работать с данными, эту базу денормализуют. Данные из десятков таблиц `Клиенты`, `Счета`, `Транзакции`, `Отделы` и `Регионы` объединяют в одну большую таблицу, или **витрину данных**. Из-за этого данные дублируются, например, название региона повторяется для каждой транзакции в этом регионе.

**Витрина данных** — это денормализованная таблица, которая позволяет быстро выполнять конкретные аналитические запросы. Она объединяет данные из нескольких нормализованных таблиц и допускает дубли в данных.

**Как инженеры данных создают витрины:**
1. Соединяют данные из нормализованных таблиц.
2. Выполняют расчёты.
3. Сохраняют результаты в отдельную таблицу

Например, из таблиц `car`, `class` и `driver` можно создать витрину `car_report`, в которой данные о классе и водителе дублируются в каждой строке `car_report`, но СУБД работает очень быстро. На текущем этапе достаточно представления того, как работают и из чего состоят витрины данных. Более подробно об этом я распишу в конспектах следующего модуля.

## Реляционная модель данных 3/5

На предыдущем слайде нас просили подумать, как вывести информацию о нескольких студентах и предметах, на которые они записаны. Давайте переведём этот вопрос на нашу базу `cars`: как вывести информацию о нескольких автомобилях и их классах? Правильный ответ — использовать операцию `JOIN`, которая связывает таблицы по ключу. Чтобы посмотреть, как это работает, выполните последнюю ячейку с кодом выше. Но если бы хотели денормализовать эти данные, нам нужно было бы создать витрину, где поля `make`, `year`, `capacity` и `transmission` уже лежат в одной таблице. Тогда бы нам не пришлось выполнять `JOIN` при каждом запросе.

## Реляционная модель данных 4/5

**Введем новое определение:**

**Отношение, или relation,** — это строгая математическая структура, которая лежит в основе таблицы. Она представляет собой подмножество декартова произведения нескольких наборов допустимых значений. Поэтому модель и называется "реляционной", так как имеет в своей основе реляционную алгебру.

**Реляционная алгебра** — это система операций над таблицами реляционной базы данных.

**Как работает реляционная алгебра в базах данных:**
1. Принимает одну или несколько таблиц на вход.
2. Выполняет одну или несколько операций с таблицами.
3. Возвращает новую таблицу на выход.

Результат любой операции — это таблица. Это позволяет комбинировать операции в цепочки и создавать сложные запросы из простых. Например, если в нашей базе данных `car` и `class` — отношения, то их соединение по `clsid` — это операция реляционной алгебры. Её результат — новое отношение, где каждая строка содержит данные об автомобиле и его классе. Чтобы посмотреть, как это работает, выполните последнюю ячейку с кодом выше.

**Например, представим два множества:**
1. D1 — все возможные марки автомобилей: `Toyota`, `BMW`, `Ford`.
2. D2 — все возможные года выпуска: 1990, 2000, 2020.

Их декартово произведение — это таблица всех возможных пар. Например, | `Toyota`, 1990, | `Toyota`, 2000 | , ... | `BMW`, 1990 | `BMW`, 2000 | и так далее. В нашей базе данных таблица `car` — это отношение. Она содержит не все возможные пары, а только те, которые существуют в базе данных. Показать полное декартово произведение на реальной базе данных практически невозможно, потому что оно абстрактно.

**Математический фундамент реляционной модели данных задаёт три ключевых правила:**
1. Строки не могут дублироваться из-за первичного ключа.
2. Порядок строк не имеет значения.
3. Значения в ячейках являются атомарными, то есть их нельзя поделить на части.

## Реляционная модель данных 5/5

Помимо схемы, реляционная модель включает **язык обработки данных, или SQL,** и **ограничения целостности**. Начнём с SQL.

**SQL, или Structured Query Language,** — это декларативный язык программирования, который используют, чтобы работать с реляционными базами данных. Каждый SQL-запрос включает в себя минимум одну операцию реляционной алгебры. Давайте рассмотрим их все на примере нашей базы данных `cars`:

- `SELECT` — выбирает указанные столбцы из таблицы и отбрасывает остальные;

In [ ]:
%%sql
SELECT make, year FROM car;

- `WHERE` — отбирает строки, которые удовлетворяют какому-либо условию;

In [ ]:
%%sql
SELECT * FROM car WHERE mileage < 50000;

- `JOIN`, `INNER JOIN`, `LEFT JOIN` — объединяет строки из двух таблиц на основании связи между столбцами;

In [ ]:
%%sql
SELECT c.make, cl.capacity
FROM car c
INNER JOIN class cl ON c.clsid = cl.clsid

- `FROM` — возвращает все возможные комбинации строк из двух таблиц;

In [ ]:
%%sql
SELECT * FROM car, class

- `UNION` — объединяет результаты двух запросов с одинаковым количеством и типом столбцов, а также удаляет дубликаты строк;

In [ ]:
%%sql
SELECT make FROM car WHERE year > 2015
UNION
SELECT make FROM car WHERE mileage < 30000;

- `INTERSECT` — возвращает только те строки, которые есть в результатах обоих запросов;

In [ ]:
%%sql
SELECT make FROM car WHERE year > 2015
INTERSECT
SELECT make FROM car WHERE mileage < 30000;

- `EXCEPT` — возвращает строки из первого запроса, которых нет во втором запросе;

In [ ]:
%%sql
SELECT make FROM car WHERE year > 2015
EXCEPT
SELECT make FROM car WHERE mileage > 30000;

- `AS` — присваивает временные имена таблицам и столбцам в рамках запроса;

In [ ]:
%%sql
SELECT
    c.make AS марка_авто,
    c.year AS год_выпуска,
    cl.capacity AS вместимость
FROM car AS c
JOIN class AS cl ON c.clsid = cl.clsid

- `GROUP BY` — выполняет вычисления над группой строк;

In [ ]:
%%sql
SELECT make, AVG(mileage) AS средний_пробег
FROM car
GROUP BY make;

**Ограничение целостности** — это набор правил, которые гарантируют корректность и надежность данных в базе. СУБД автоматически проверяет, чтобы эти правила соблюдались.

**В нашей базе данных `cars` есть несколько ограничений:**
1. **Первичный ключ.** Он гарантирует, что каждая строка в таблице уникальная. Например, столбец `cid` в таблице `car` должен иметь уникальное значение для каждого автомобиля. Это позволяет избежать дублей в записях.
2. **Внешний ключ.** Он гарантирует, что связь между таблицами является целостной. Например, значение в столбце `clsid` таблицы `car` должно существовать в столбце `clsid` таблицы `class`. Это не позволяет ссылаться на несуществующий класс.
3. **Проверочное ограничение.** Оно гарантирует, что данные попадают в заданный диапазон или соответствуют какому-то конкретному условию. Например, можно задать правило, что `year > 1900` для столбца года выпуска.

## Терминология и обозначения

**Повторюсь и сделаю акцент на том, что каждый элемент реляционной модели имеет два названия:**
1. **Теоретическое** — из реляционной алгебры.
2. **Практическое** — из SQL и баз данных.

**Изучите небольшую табличку, чтобы отстроить термины и лучше их запомнить:**

| Теоретический термин из реляционной алгебры | Практический термин из SQL и баз данных | Пример из базы данных `cars` | Обозначение |
|---|---|---|---|
| Отношение, Relation | Таблица, Table | `car`, или `class`, или `driver` | `R(A1, A2, ...)` |
| Кортеж, Tuple | Строка, или Запись, Row | `1, 'renault', 2015, 35000, 'D'` | `t ∈ R` |
| Атрибут, Attribute | Столбец, или Поле, Column | `cid`, или `make`, или `year`, или `mileage` | `A1, A2, ...` |
| Заголовок, Header | Схема таблицы, Schema | `car(cid, make, year, mileage, clsid)` | `H(R)` |
| Тело, Body | Данные таблицы, Data | Все строки в таблице `car` | `B(R)` |

## Таблица (отношение) в РМД

Реляционная модель требует точно описывать каждое отношение. Такая формальная запись следует строгой последовательности и включает **три ключевых аспекта:**

1. **Формальная запись отношения.** Например, `car(cid, make, year, mileage, clsid)`.
    Эта запись означает следующее:
    - `car` — название отношения;
    - `(cid, make, year, mileage, clsid)` — заголовок отношения;
    - `cid` -> `make` -> `year` — фиксированный порядок атрибутов.

2. **Состав отношения.** Каждое отношение состоит из двух частей:
    - заголовок — определяет имена и порядок атрибутов таблицы;
    - тело — содержит сами данные в виде набора кортежей.

Например, в нашей базе данных `cars` отношение `class` имеет заголовок `(clsid, capacity, bags, transmission)`, а тело отношения `class` содержит конкретные строки со значениями этих атрибутов.

3. **Количественные характеристики.** Для любого отношения можно вычислить две основные характеристики:
    - **степень отношения** — это количество атрибутов в заголовке;
    - **мощность отношения** — это количество кортежей в теле.

**Давайте посмотрим, какую степень и мощности имеют отношения в нашей базе даных `cars`.**

Начнём с отношения `car`:

In [ ]:
%%sql
SELECT 
    'car' AS отношение,
    COUNT(*) AS степень,
    (SELECT COUNT(*) FROM car) AS мощность,
    'car(' || STRING_AGG(column_name, ', ' ORDER BY ordinal_position) || ')' AS формальная_запись
FROM information_schema.columns 
WHERE table_name = 'car' AND table_schema = 'public';

Отношение `car` содержит **6** атрибутов: `cid`, `make`, `year`, `mileage`, `clsid` и `res_number`. В базе хранит **7** автомобилей. Формальная запись `car(cid, make, year, mileage, clsid, res_number)` задаёт порядок атрибутов. Система применяет этот порядок, когда выбирает данные или соединяет таблицы.

Продолжим отношением `class`:

In [ ]:
%%sql
SELECT 
    'class' AS отношение,
    COUNT(*) AS степень,
    (SELECT COUNT(*) FROM class) AS мощность,
    'class(' || STRING_AGG(column_name, ', ' ORDER BY ordinal_position) || ')' AS формальная_запись,
    STRING_AGG(column_name || ' (' || data_type || ')', ', ' ORDER BY ordinal_position) AS атрибуты_с_типами
FROM information_schema.columns 
WHERE table_name = 'class' AND table_schema = 'public';

Отношение `class` содержит **4** атрибута: `clsid`, `capacity`, `bags` и `transmission`. В базе хранит **4** класса автомобилей. Формальная запись `class(clsid, capacity, bags, transmission)` задаёт порядок атрибутов. Запросы применяют эту структуру, когда соединяют таблицы `car` и `class` по полю `clsid`.

Теперь отношение `driver`:

In [ ]:
%%sql
SELECT 
    'driver' AS отношение,
    COUNT(*) AS степень,
    (SELECT COUNT(*) FROM driver) AS мощность,
    'driver(' || STRING_AGG(column_name, ', ' ORDER BY ordinal_position) || ')' AS формальная_запись,
    STRING_AGG(column_name || ' (' || data_type || ')', ', ' ORDER BY ordinal_position) AS атрибуты_с_типами
FROM information_schema.columns 
WHERE table_name = 'driver' AND table_schema = 'public';

Отношение `driver` содержит **3** атрибута: `did`, `name` и `gender`. В базе хранит **7** водителей. Формальная запись `driver(did, name, gender)` задаёт порядок атрибутов. Система использует эту структуру, когда ищет водителей по имени или связывает их с записями об аренде.

Затем отношение `res`:

In [ ]:
%%sql
SELECT 
    'res' AS отношение,
    COUNT(*) AS степень,
    (SELECT COUNT(*) FROM res) AS мощность,
    'res(' || STRING_AGG(column_name, ', ' ORDER BY ordinal_position) || ')' AS формальная_запись,
    STRING_AGG(column_name || ' (' || data_type || ')', ', ' ORDER BY ordinal_position) AS атрибуты_с_типами
FROM information_schema.columns 
WHERE table_name = 'res' AND table_schema = 'public';

Отношение `res` содержит **8** атрибутов: `bid`, `start`, `finish`, `days`, `pickat`, `cid`, `did`, `isvip`. В базе хранит **9** записей об аренде. Формальная запись `res(bid, start, finish, days, pickat, cid, did, isvip)` задаёт порядок атрибутов. СУБД применяет эту структуру, когда рассчитывает длительность аренды или фильтрует VIP-бронирования.

## Таблица (отношение) в базе данных

**Реляционная модель предъявляет пять требования к каждому отношению:**
1. **Столбцы должны иметь уникальные названия.** В одном отношении не может быть двух столбцов с одинаковым названием. Например, в таблице `car` все имена столбцов различаются: `cid`, `make`, `year`, `mileage`, `clsid`.
2. **Каждая ячейка таблицы должна иметь только одно значение**. Нельзя хранить в одной ячейке список или массив значений. Например, в столбце `make` может быть значение `renault`, но не может быть сразу несколько значений: `renault`, `ford`, `bmw`.
3. **Данные записываются в виде неупорядоченных кортежей.** Строки в таблице не имеют порядка. Система может может возвращать строки в любом порядке, если не указывать сортировку явно.
4. **Все кортежи уникальны**. В одном отношении не может быть полностью двух одинаковых строк. Это правило гарантирует первичный ключ.
5. **Одному столбцу соответствуе один тип данных.** Все значения в одном столбце имеют одинаковый тип. Например, в столбце `year` таблицы `car` все значения — это целые числа, а в столбце `make` — текст.

Реляционная модель определяет только то, как хранятся данные в рамках логической структуры. Она не учитывает, как система физически хранит данные на диске, как индексирует их, или как оптимизирует доступ. Это позволяет разным СУБД, например, PostgreSQL, MySQL и SQLite хранить данные по-своему, но сохранять единый интерфейс через SQL. 

Эти требования делают работу с данными предсказуемой и надёжной. Они гарантирую, что запросы всегда возвращают цельные результаты, а структура данных остаётся управляемой независимо от масштаба системы.

Каждое отношение в базе данных соответствует конкретному типу объектов из реального мира и описывает их свойства. Например, в нашей базе данных `cars` `class(clsid, capacity, bags, transmission)` — это класс автомобиля. Он описывает категории автомобилей с их характеристиками: вместимостью, багажником и типом трансмиссии. Кроме того, в базе данных `cars` есть и класс `driver(did, name, gender)` — это класс водителя. Он описывает людей, которые арендуют автомобили: их идентификатор, имя и пол. 

**Как отношения моделируют классы объектов:**

Посмотрим на примере отношения `class`. Выполните ячейку кода ниже. Результат покажет конкретные экземпляры класса.

In [ ]:
%%sql
SELECT * FROM class;

Результат показывает четыре конкретных класса автомобилей. Каждая строка содержит значения четырёх атрибутов: идентификатор класса, вместимость, количество сумок и тип трансмиссии. Например, класс A — это автомобиль на 2 человека с 1 сумкой и механической коробкой передач.

А теперь изучим пример с отношением `driver`:

In [ ]:
%%sql
SELECT * FROM driver;

Результат показывает семь конкретных водителей. Каждая строка содержит значения трёх атрибутов: идентификатор водителя, имя и пол. Например, водитель с идентификатором 1 — это Jared мужского пола.

## Таблица (отношение) обычно связана с объектами реального мира и их атрибутами

Реляционная модель отражает связи между реальными объектами через связи между таблицами. Каждая таблица соответствует одному типу сущеностей, а связи между таблицами показывают, как эти сущности взаимодействуют в реальном мире. Например, в базе `cars` отношения `driver`, `car` и `class` описывают разные сущности. Отношение `res` фиксирует факт аренды и связывает их.

**У нас есть четыре таблицы:**
1. Таблица `driver` — содержит данные о водителях.
2. Таблица `car` — содержит данные об автомобилях.
3. Таблица `class` — содержит данные о классах автомобилей.
4. Таблица `res` — записывает факт аренды и связывает водителей с автомобилями.

Выполните запрос ниже, чтобы увидеть, как он связывает четыре таблицы и моделирует связи между объектами реального мира. Этот запрос позволяет нам узнать, какие автомобили какого класса арендовал конкретный водитель.

In [ ]:
%%sql
SELECT 
    d.name AS водитель,
    c.make AS автомобиль,
    cl.capacity AS вместимость,
    cl.transmission AS трансмиссия
FROM res r
JOIN driver d ON r.did = d.did
JOIN car c ON r.cid = c.cid
JOIN class cl ON c.clsid = cl.clsid
LIMIT 5;

# Первичный ключ (1/2) + (2/2)

**Ключ-кандидат** — это набор атрибутов, который определяет каждую строку в отношении. Ключ должен быть минимальным, то есть мы не сможем удалить из него ни один атрибут без потери уникальности.

Например, в отношении `driver` `did` — это ключ-кандидат, потому что он уникально идентифицирует водителя, а `name` и `gender` не являются ключами-кандидатами, так как в нашем отношении могут быть тёзки одного пола.

Первичный ключ, о котором мы говорили выше, это ключ-кандидат, который аналитик выбирает в качестве основного идентификатора строк. Первичный ключ позволяет ускорить поиск строк. СУБД автоматически создаёт индекс по первичному ключу. Когда мы ищем строку по значению ключа, система использует индекс и находит данные быстро, без перебора всех строк.

Например, ячейка ниже выполнится быстро, даже если в таблице есть миллионы строк. Это произойдёт потому, что поиск по первичному ключу `cid` использует индекс:

In [ ]:
%%sql
SELECT * FROM car WHERE cid = 5;

Если первичный ключ состоит из нескольких столбцов, то он называется составным. Для того, чтобы поиск в таблице был максимально быстрым, он должен использовать все столбцы ключа.

Что касается аналитических баз данных, таких как хранилища данных, там часто первичные ключи не используют. Это происходит потому, что в DWH важнее скорость загрузки больших объёмов данных и их агрегация, а не уникальность каждой строки в реальном времени.

## Концепция внешнего ключа (1/3) + (2/3) + (3/3)

**Внешний ключ** — это атрибут в одной таблице, который ссылается на первичный ключ в другой таблице. Он создаёт связь между строками разных отношений. Например, в нашей базе данных `cars` таблица `car` содержит столбец `clsid`. Этот столбец ссылается на первичный ключ `clsid` в таблице `class`. Каждый автомобиль принадлежит к одному классу.

Выполните ячейку ниже, чтобы увидеть, как `clsid` в `car` ссылается на `clsid` в `class`:

In [ ]:
%%sql
SELECT 
    c.cid AS id_автомобиля,
    c.make AS марка,
    c.clsid AS код_класса,
    cl.capacity AS вместимость,
    cl.transmission AS трансмиссия
FROM car c
JOIN class cl ON c.clsid = cl.clsid
LIMIT 5;

Результат показывает пять конкретных автомобилей с данными их классов. Каждая строка содержит марку автомобиля, код его класса, вместимость и тип трансмиссии — данные, которые система берёт из двух связанных таблиц. Внешний ключ гарантирует, что автомобиль нельзя отнести к несуществующему классу, позволяет получать данные из обеих таблиц одним запросом и сохраняет целостность данных, когда мы удаляем или изменяем классы. То есть внешний ключ в дочерней таблице `car` строго ссылается на первичный ключ в родительской таблице `class`. Давайте разберёмся, как работает эта связь.

В нашей базе `cars` родительская таблица, то есть `class` содержит первичный ключ `clsid`, а дочерняя таблица `car` содержит внешний ключ `clsid`, который ссылается на `class.clsid`. Ключевые правило состоит в том, что внешний ключ может ссылаться только на столбец, который является первичным ключом или имеет ограничение уникальности, или UNIQUE constaint. То есть система не разрешит создать ссылку на неуникальный столбец, потому что не сможет понять, на какую строку ссылаться, когда пользователь напишет соответствующий запрос. Давайте проверим, что `clsid` в `class` является первичным ключом. Для этого выполните ячейку ниже.

In [ ]:
%%sql
SELECT 
    'Родительская таблица: class' AS роль,
    'Первичный ключ: ' || STRING_AGG(column_name, ', ') AS ключ
FROM information_schema.key_column_usage 
WHERE table_name = 'class' 
    AND constraint_name LIKE '%pkey%'
UNION ALL
SELECT 
    'Дочерняя таблица: car' AS роль,
    'Потенциальный FK: clsid → class.clsid' AS ключ;

Так, столбец `clsid` в таблице `class` является первичным ключом, а в таблице `car` столбец `clsid` может выступать как внешний ключ. Поэтому система позволяет создать внешний ключ из `car.clsid` на `class.clsid`. Это защищает таблицу от логических ошибок и гарантирует, что каждый автомобиль ссылается на однозначно определённый класс. Без первичного ключа в `class` такая связь была бы невозможной.

## Поддержка ссылочной целостности в РМД (1/2)

Система не только устанавливает связи через внешние ключи, но и строго контролирует их целостность. Сервер базы данных автоматически проверяет, есть ли нарушения связей между данным, если обнаруживает их, то исправляет.

**Изучите три ограничения контроля целостности:**

1. **Система запрещает вставлять некорректную ссылку.** Она отклоняет запись, если значение внешнего ключа не существует в родительской таблице. Давайте попробуем вставить в наше отношение `car` автомобиль с несуществующим классом Z. Выполните ячейку ниже, чтобы проверить, как это работает.

In [ ]:
%%sql
INSERT INTO car (cid, make, year, mileage, clsid) 
VALUES (100, 'TestCar', 2023, 0, 'Z');

Система не дала создать запись в таблице `car`. Она обнаружила, что значение `clsid`='Z' отсутствует в таблице `class`. Это показывает, как внешний ключ защищает от ссылок на несуществующие данные. Система автоматически проверяет целостность при вставке.

2. **Система запрещает удалять запись, на которую есть ссылки.** Она не позволит удалить какой-либо класс, если на него, например, ссылается другой. А теперь попробуем удалить класс `A`, на который ссылаются автомобили.

In [ ]:
%%sql
SELECT cid, make, clsid FROM car WHERE clsid = 'A';
DELETE FROM class WHERE clsid = 'A';

Система заблокировала удаление записи из таблицы `class`. Она указала, что на класс `clsid='A'` всё ещё ссылаются строки в таблице `car`. Это показывает, как внешний ключ предотвращает удаление данных, которые используются в других таблицах. Сначала нужно удалить или изменить все зависимые записи.

3. **Система запрещает менять ключ, на который есть ссылки.** Она не позволяет менять значение первичного ключа, если на него ссылаются другие таблицы. Теперь давайте попробуем изменить `clsed` класса `A` на `Z`, когда на него есть ссылка.

In [ ]:
%%sql
UPDATE class SET clsid = 'Z' WHERE clsid = 'A'

Система отклонила попытку изменить значение первичного ключа `clsid` с `A` на `Z`. Она сообщила, что на исходное значение всё есть ссылки из таблицы `car`. Это показывает, как внешний ключ защищает не только от удаления, но и от изменений ключевых значений, на которые ссылаются другие данные.

Система проверяет дополнительные условия, которые устанавливает триггер. Если целостность связей в базе данных `cars` будет нарушена, то нарушатся и бизнес-правила, которые задают требования. Давайте проверим, все ли `clsid` в таблице `car` должны существовать в отношении `class`.

In [ ]:
%%sql
SELECT 
    CASE 
        WHEN EXISTS (
            SELECT 1 FROM car 
            WHERE clsid NOT IN (SELECT clsid FROM class)
        )
        THEN 'Есть нарушения целостности'
        ELSE 'Все ссылки корректны'
    END AS проверка_целостности;

Мы выяснили, что ссылки корректны, а связи целы. Такие проверки работают автоматически. Когда пользователь пытается нарушить целостность, система сразу отклоняет операцию с ошибкой. Это позволяет защитить данные.

## Поддержка ссылочной целостности в РМД (2/2)

Сейчас система блокирует операции, которые нарушают связи между таблицами. Но можно задать другие правила. Для этого используют правила `ON DELETE` и `ON UPDATE`, когда создают внешний ключ. Они говорят системе, что делать в двух ситуациях:
1. **`ON DELETE`** — что делать, когда пытаются удалить запись из родительской таблицы, на которую есть ссылки.
2. **`ON UPDATE`** — что делать, когда пытаются изменить значение первичного ключа в родительской таблице, на который есть ссылки

Давайте посмотрим, какие правила действуют сейчас для связи `car` -> `class`:

In [ ]:
%%sql
SELECT 
    confdeltype AS код_правила_удаления,
    confupdtype AS код_правила_обновления
FROM pg_constraint
WHERE conrelid = 'car'::regclass AND contype = 'f';

Как вы видите, мы получили код `a`, или `NO ACTION`. Это значит, что система сейчас запрещает операции.

**Для каждой ситуации мы можем выбрать одно из трёх действий:**
1. **`CASCADE`** — система повторяет ту же операцию в дочерней таблице.
2. **`SET NULL`** — система заменяет значение внешнего ключа на `NULL` или заданное значение по умолчанию.
3. **`RESTRICT` / `NO ACTION`** — система отменяет операцию и возвращает ошибку. Это текущая настройка в нашей базе данных.

Например, если бы мы определили связь как `FOREIGN KEY (clsid) REFERENCES class(clsid) ON DELETE CASCADE` то при удалении класса система автоматически удалила бы все автомобили этого класса. Это позволяет точно моделировать бизнес-логику: удаление категории товаров может требовать удаления всех товаров этой категории.

## Графический интерфейс - DBeaver CE

Чтобы удобно работать с различными СУБД, в том числе и PostgreSQL в нашем курсе, используют клиент **[DBeaver Community Edition](https://dbeaver.io/download/)**, as known as "Бобёр". Эта программа предоставляет пользователю графический интерфейс вместо командной строки. DBeaver можно использовать также и с MySQL, SQLite, Oracle и другими СУБД. Это позволяет работать с разными базами данных в одной программе. Интерфейс автоматически определяет тип СУБД и предоставляет необходимые инструменты. Кроме того, он позволяет проще переносить данные между разными форматами и базами. Например, вы можете импортировать данные из CSV-, Excel-, JSON-файлов в таблицы базы данных данных и экспортировать результаты запросов в различные форматы.

## Графический интерфейс - pdAdmin

Чтобы работать конкретно с СУБД PostgreSQL иногда используют клиент **[pgAdmin](https://www.pgadmin.org/)**. Он также предоставляет графический интерфейс и существует как самостоятельное приложение. В отличие от универсальных клиентов, pgAdmin создан специально для PostgreSQL. Это означает полную поддержку всех специфичных для этой СУБД функций, типов данных и команд.

Я не буду подробно расписывать обо всех клиентах, которые позволяют работать с СУБД, а также опущу информацию о том, как подключаться с помощью командной строки. Информацию об этом вы можете найти в Интернете.

# Введение в SQL DDL, или Data Definition Language

Ранее мы рассмотрели инструменты для работы с базами данных. Теперь рассмотрим детальнее язык, на котором мы взаимодействуем с реляционной моделью. Мы знаем, что язык структурированных запросов, или SQL, позволяет описывать данные, запрашивать информацию и управлять доступом. Давайте узнаем о нём чуть больше.

## Язык структурированных запросов

Первый официальный стандарт SQL приняли в 1986 году. Это был ANSI SQL-86. Этот стандарт стал единой основной для всех СУБД. Текущая версия стандарта — SQL:2023. Развитие стандарта означает, что основные конструкции языка остаются стабильными, но появляются новые возможности для работы с современными типами данных.

**В язык SQL входят несколько групп команд, каждая из которых решает свою задачу. Всего таких групп выделяют четыре:**

1. **DDL, или Data Definition Language,** — это язык описания данных. Команды в нём позволяют создавать, изменять и удалять структуры данных, например, таблицы, индексы и представления. Это `CREATE TABLE`, `ALTER TABLE`, `DROP TABLE`, `TRUNCATE TABLE`.
2. **DML, или Data Manipulation Language,** — это язык работы с данными. Команды в нём позволяют выбирать, добавлять, удалять и обновлять строки. Это `SELECT`, `INSERT`, `UPDATE` и `DELETE`.
3. **DCL, или Data Control Language,** — это язык контроля доступа. Команды в нём разрешают или запрещают доступ к данным и операциям. Это `GRANT` и `REVOKE`.
4. **TCL, или Transaction Control Language,** — это язык управления транзакциями. Команды в нём позволяют управлять логическими блоками операций и обеспечивают их целостность. Это `COMMIT`, `ROLLBACK` и `SAVEPOINT`.

**Такое разделение позволяет чётко отделить этапы работы:**
1. Cначала проектируют структуру — DDL.
2. Затем наполняют её данными и работают с ними — DML, а также параллельно контролируют доступ — DCL и гарантируют надёжность операций — TCL.

Это делает процессы разработки и администрирования предсказуемыми.

База данных — это не только таблицы. Это система связанных объектов, которые организуют данные, задают правила работы и контролируют доступ. Рассмотрим ключевые объекты, с которыми работает система.

## Объекты базы данных, с которыми мы будем работать

**В базе данных есть семь основных объектов:**
1. **Схема, или `schema`** — это логический контейнер. Схема группирует другие объекты, например, таблицы и представления, внутри базы данных. Все наши таблицы `car`, `class`, `driver`, `res` находятся в схеме по умолчанию — `public`. Схемы разделяют пространство имён и помогают организовать структуру.
2. **Таблица, или `table`** — основной объект для хранения данных. Таблица имеет строгую структуру: фиксированный набор столбцов и произвольное количество строк. Именно таблицы мы анализировали ранее на примере автомобилей и водителей.
3. **Представление, или `view`** — виртуальная таблица, которая не хранит данные самостоятельно. Представление определяет запрос `SELECT`. Когда вы обращаетесь к представлению, система выполняет этот запрос и возвращает результат как таблицу. Это удобно для сложных, часто используемых выборок.
4. **Колонка, или `column`** — отдельное поле в таблице или представлении. Каждая колонка имеет уникальное имя в пределах таблицы и определённый тип данных, например, `text` для `car.make` или `integer` для `car.year`.
5. **Ограничение, или `constraint`** — правило, которое система проверяет для данных. Примеры ограничений: `PRIMARY KEY` — первичный ключ, `FOREIGN KEY` — внешний ключ, `UNIQUE` — уникальность, `CHECK` — проверка условия, `NOT NULL` — запрет на пустые значения. Ограничения поддерживают целостность данных.
6. **Функция, или `function`, и процедура, или `procedure`** — это программные блоки. Функции и процедуры содержат код на SQL или других языках, например, PL/pgSQL. Они выполняют вычисления, преобразуют данные или реализуют бизнес-логику. Функция возвращает результат, процедура — нет.
7. **Роль, или `role`** — объект для управления доступом. Роль представляет пользователя базы данных или группу пользователей. Привилегии на доступ к таблицам, схемам или другим объектам назначают именно ролям.

Все эти объекты связаны между собой. Схема содержит таблицы и представления. Таблицы состоят из колонок. Ограничения прикрепляют к конкретным таблицам или колонкам. Функции и процедуры также принадлежат схемам. Роли получают права на выполнение действий с этими объектами.

Мы рассмотрели, из каких объектов состоит база данных. Теперь узнаем, как эти объекты создают на практике. Для этого можно использовать не только текстовые SQL-команды, но и графические интерфейсы, такие как DBeaver CE или pgAdmin.

## DDL и инструменты с графическим интерфейсом

**Как графический интерфейс работает с DDL:**
1. Вместо того, чтобы писать код вручную, вы можете использовать визуальный редактор. Например, в DBeaver CE вы кликаете правой кнопкой на схеме, выбираете «Создать таблицу» и заполняете форму: указываете имя таблицы, добавляете колонки, выбираете типы данных и задаёте ограничения — всё через диалоговые окна и списки.
2. Когда вы нажимаете «Сохранить» в графическом интерфейсе, программа автоматически генерирует точный SQL-скрипт на языке DDL — тот самый, который вы могли бы написать сами. Затем этот скрипт программа незаметно для вас отправляет на сервер PostgreSQL, чтобы выполнить его.
3. Перед финальным сохранением изменений DBeaver CE часто показывает окно с предварительным просмотром сгенерированного SQL-запроса. Это позволяет проверить, что именно система выполнит. Например, вы можете увидеть следующую готовую команду:

In [ ]:
%%sql
CREATE TABLE public.new_car (
    id integer PRIMARY KEY,
    model text NOT NULL,
    year integer
);

Графический интерфейс не заменяет SQL DDL, а делает работу с ним нагляднее. Он помогает избежать синтаксических ошибок в ручном наборе и ускоряет проектирование структуры базы, особенно, когда есть множество связанных таблиц и ограничений. При этом понимание сгенерированного скрипта позволяет точно контролировать результат и в дальнейшем воспроизводить его, например, чтобы развернуть базу на другом сервере.

## Работа с базой данных через клиентское ПО

После того как мы узнали, как создавать объекты, нужно понять, как подключиться к серверу и работать с существующей базой. Графические клиенты, такие как DBeaver CE, требуют для этого параметры соединения.

Чтобы установить соединение, вы должны указать клиенту несколько параметров. Эти параметры включают адрес сервера PostgreSQL, например, `localhost` или IP-адрес, порт подключения — обычно `5432`, имя конкретной базы данных — `cars`, а также ваше имя пользователя и пароль. Без этих данных клиент не сможет найти сервер и получить доступ.

Когда соединение установлено, клиент отображает структуру базы в виде дерева. Эта иерархия отражает логическую организацию данных:
1. **Сервер** — верхний уровень. Представляет собой экземпляр PostgreSQL, который может содержать множество баз данных.
2. **База данных** — следующий уровень внутри сервера. Наша учебная база называется `cars`.
3. **Схема** — контейнер внутри базы данных, который группирует логически связанные объекты. По умолчанию используется схема `public`.
4. **Таблица** — основной объект внутри схемы, который физически хранит данные в виде строк и столбцов. В схеме `public` находятся таблицы `car`, `class`, `driver`, `res`.
5. **Представление** — виртуальная таблица внутри схемы. Представление не хранит данные самостоятельно, а предоставляет доступ к данным через заранее определённый запрос `SELECT`. Обычно оно основывается на одной или нескольких реальных таблицах.

Вы подключаетесь к серверу PostgreSQL с базой `cars`. В дереве объектов DBeaver CE вы раскрываете ветку базы `cars`, затем схему `public`. Внутри вы видите список всех таблиц и представлений. Кликая на таблицу `car`, вы можете просматривать её данные, а кликая на вкладку «Ограничения» — видеть привязанные к ней правила, такие как `PRIMARY KEY` или `FOREIGN KEY`. Эта визуальная навигация позволяет быстро находить нужный объект и изучать его свойства без сложных команд.

Графические интерфейсы упрощают работу, но основа управления структурой данных — язык SQL, а точнее, его часть DDL. Когда вам нужно точно контролировать процесс или автоматизировать создание объектов, вы пишете DDL-запросы напрямую.

## Как создать таблицу с помощью SQL?

Ключевая команда для создания новой таблицы — `CREATE TABLE`. Её структуру задают так:

In [ ]:
%%sql
CREATE TABLE имя_таблицы (
    описание_поля_1,
    описание_поля_2,
    ...,
    ограничение_1,
    ограничение_2
);

Команда начинается с ключевых слов `CREATE TABLE`, за которыми следует имя новой таблицы. Всё описание таблицы заключают в круглые скобки. Внутри этих скобок через запятую перечисляют описания колонок, или полей, и ограничений целостности.

Каждое поле в таблице определяют **минимум двумя** элементами, которые указывают последовательно:
1. **Название поля, `field_name`,** — уникальный идентификатор колонки в пределах таблицы.
2. **Тип данных, `data_type`,** — определяет, какие значения может хранить колонка. Например, `INTEGER` для целых чисел, `TEXT` для строк, `DATE` для дат.

Например, запись `model TEXT` создаёт колонку с именем `model`, которая будет хранить текстовые данные.

После обязательной части можно указать необязательные атрибуты, которые уточняют поведение колонки:

- **`NULL` или `NOT NULL`** — указывает, может ли поле содержать пустое значение `NULL`. По умолчанию используется `NULL`.

- **`DEFAULT` <value>** — задаёт значение по умолчанию, которое система подставит, если при вставке новой строки для этого поля не указали явное значение.

Таким образом, полное описание колонки может выглядеть так: `mileage INTEGER DEFAULT 0`. Это создаст поле `mileage` для целых чисел, которое, если значение не указано явно, автоматически получит значение **0**, что логично для пробега нового автомобиля. Или `created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP`, где система автоматически проставит дату и время создания записи.

## Наименования объектов в Postgres

После определения структуры таблицы важно правильно назвать её и её колонки. Согласованные правила именования предотвращают ошибки, упрощают чтение кода и команд, а также обеспечивают единообразие в работе всей команды разработки.

Перед тем, как проектировать базы данных все участники команды должны договориться о единых правилах. Это касается имён таблиц, колонок, схем, ограничений. Единый стиль избавляет от путаницы, когда один разработчик пишет `CustomerName`, другой — `customer_name`, а система может считать эти имена разными объектами.

Для имён таблиц и колонок в PostgreSQL рекомендуют использовать стиль snake_case. Этот стиль использует только строчные буквы латинского алфавита, цифры и символ подчёркивания _, чтобы разделять слова. Например: `car`, `driver`, `start_date`, `reservation_number`.

Если вы используете заглавные буквы в имени, например `Building_Id`, PostgreSQL потребует заключать такое имя в двойные кавычки во всех запросах: `SELECT "Building_Id" FROM table;`. Без кавычек система автоматически приведёт имя к нижнему регистру и будет искать колонку `building_id`.

**Изучите два ключевых правила регистра:**
- имена без кавычек система неявно преобразует к нижнему регистру. Поэтому `building_id`, `Building_Id` и `BUILDING_ID` для PostgreSQL — это одно и то же имя.

- имена в двойных кавычках система сохраняет в исходном регистре. Поэтому `"Building_Id"` и `"building_id"` — это два разных имени объектов. Лучше избегать кавычек, чтобы не допускать ошибок

Соблюдайте стиль `snake_case` и используйте только строчные буквы. Называйте таблицы `client_order`, а не `ClientOrder`, колонки — `phone_number`, а не `phoneNumber`. Это гарантирует, что, например, запрос `SELECT phone_number FROM client_order` будет работать всегда, и избавит от проблем с регистром имён.

## Часто используемые свойства колонок (1/2) + (2/2)

Система управления базами данных позволяет задавать для колонок специальные свойства. Два самых распространённых из них — это счётчик `GENERATED AS IDENTITY` и значение по умолчанию `DEFAULT`:

1. **Счётчик `GENERATED AS IDENTITY`** — автоматически генерирует новое уникальное числовое значение каждый раз, когда вы вставляете новую строку в таблицу. Он позволяет удобно создавать первичные ключи
    
    Представьте, что нам нужно создать таблицу `driver`. Мы хотим, чтобы у каждого водителя был уникальный числовой идентификатор, который система присваивает автоматически. Без счётчика нам пришлось бы вручную вычислять и передавать следующий свободный `id` в каждом запросе `INSERT`. Код, который использует счётчик, выглядит так:

In [ ]:
%%sql
CREATE TABLE driver (
    driver_id int GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name text NOT NULL,
    gender text NOT NULL
);

Этот код создаёт колонку `driver_id`, которая с помощью свойства `GENERATED ALWAYS AS IDENTITY` автоматически генерирует уникальные числовые значения с **1**, и одновременно объявляет её первичным ключом таблицы.

2. **Значение по умолчанию `DEFAULT`** — позволяет задать значение, которое система подставит в колонку автоматически, если во время операции `INSERT` для этой колонки не передали явное значение.
    Рассмотрим два сценария из нашей базы данных:

- логично предположить, что новый автомобиль, только что добавленный в автопарк, имеет нулевой пробег. Мы можем задать это правило на уровне базы данных:

In [ ]:
%%sql
mileage int NOT NULL DEFAULT 0

Теперь, если выполнить `INSERT INTO car (make, year) VALUES ('Toyota', 2023)`, система автоматически запишет в колонку `mileage` значение **0**.

- часто важно знать, когда именно запись появилась в базе. Вместо того чтобы передавать текущее время из приложения, можно поручить это самой СУБД:

In [ ]:
%%sql
created_time timestamp NOT NULL DEFAULT NOW()

Функция `NOW()` возвращает текущие дату и время сервера БД. При каждой вставке новой строки система вызовет эту функцию и запишет её результат в колонку `created_time`.

Свойство `DEFAULT` сохраняет целостность данных прямо в схеме таблицы. Система гарантирует, что колонка всегда получит значение, даже если его не передали, позволяет не писать код, чтобы заполнять такие поля.

## Проверка соответствия типа значения

Сервер базы данных проверяет каждое значение, которое вы пытаетесь записать в таблицу. СУБД сверяет это значение с типом данных, который объявили для колонки. Это фундаментальное правило определяет строгость и предсказуемость того, как хранится информация.

Если вы объявили колонку `year` как `INTEGER`, система будет принимать только целые числа. Она не позволит вам записать в это поле текст, дату или дробное число.

Давайте проверим это правило на примере из базы `cars`. Попробуем обновить год выпуска автомобиля, но вместо числа передадим строку с текстом:

In [ ]:
%%sql
UPDATE car SET year = 'ABC' WHERE cid = 1;

Система сразу вернёт ошибку, например, `invalid input syntax for type integer: "ABC"`. Эта ошибка показывает механизм проверки в действии: СУБД получила значение 'ABC', распознала его как текст, сравнила с ожидаемым типом колонки `INTEGER` и отклонила операцию, потому что типы не совпадают.

**Давайте рассмотрим три следствия этого правила:**
1. **Каждая колонка имеет строго заданный тип.** Тип данных — это как контракт между разработчиком и СУБД. Когда вы объявляете поле как `DATE`, вы гарантируете, что в нём будут только даты, что делает возможным, например, сравнивать, сортировать и вычислять интервалы.

2. **Для каждого типа данных в SQL существуют свои правила записи констант.** Вы передаёте значения в запросах в виде констант, и их формат зависит от типа, например:
    - числа пишут просто `35000`;
    - cтроки заключают в одинарные кавычки: `'renault'`;
    - даты тоже часто требуют кавычек и определённого формата: `'2015-01-15'`. Система проверяет не только тип, но и валидность таких значений, например, что `'2023-13-45'` не является корректной датой.

3. **Названия типов данных могут различаться в разных СУБД.** Имена типов — часть диалекта SQL конкретной системы. Например, тип для строки текста неограниченной длины в PostgreSQL — это `TEXT`, а в Microsoft SQL Server — `VARCHAR(MAX)` или `NVARCHAR(MAX)`. Тип для логического значения в PostgreSQL — `BOOLEAN`, в SQL Server — `BIT`. Поэтому скрипт для таблицы для одной СУБД, почти всегда потребует правки для работы в другой.

## Часто используемые типы данных в Postgres

Мы убедились, что СУБД строго проверяет соответствие данных их типу. Теперь важно разобраться, какие именно типы данных чаще всего используют для работы. В PostgreSQL есть большой набор встроенных типов, каждый из которых решает свою задачу. Эти типы позволяют точно описывать свойства объектов: количество, текст, стоимость, даты или логические состояния.

**Выделим пять основных групп типов данных:**
1. **Целые числа `int` или `bigint`**. Эти типы хранят целые числа без дробной части:

    - тип `int`, или `integer` — это стандартный тип для целых чисел. Он подходит для счётчиков, возраста, года выпуска, количества. В таблице cars поле year имеет тип int.
    - тип `bigint` хранит целые числа в большем диапазоне. Его используют, когда значения могут превысить лимит типа int, например, для очень крупных идентификаторов.

2. **Текст `varchar`, `char` или `text`**. Эти типы хранят символьные строки:

    - тип `varchar` с максимальной длиной — это текст переменной длины. Система резервирует место только под фактическую длину строки.
    - тип `char` с длиной — это текст фиксированной длины. Если строка короче, система дополняет её пробелами. Его используют реже, например, для кодов фиксированного формата.
    - тип `text` — это текст без ограничения длины. Универсальный тип для хранения больших текстов: описаний, комментариев, статей.

В этой PostgreSQL нет разницы в производительности между типами `varchar`, `char` и `text`. Разработчики часто выбирают `text` для простоты или `varchar` с длиной, чтобы явно указать бизнес-ограничение.

3. **Дробные числа `numeric`, `float` или `real`.** Этипы типы хранят числа с плавающей точкой:
    - тип `numeric` с параметрами, или `decimal` — это тип для точных вычислений, где важна каждая цифра, например, деньги или процент. Параметры задают общее количество знаков и количество знаков после запятой. Например, `numeric` с параметрами **10** и **2** хранит числа вроде **99999999.99**.
    - типы `real` и `float` — это типы для приближённых вычислений, где важнее скорость и диапазон, чем абсолютная точность, например, какие-либо научные данные. Они могут накапливать ошибки округления.

4. **Дата и время `date`, `timestamp`, `timestamptz` или `interval`.** Эти типы хранят дату или промежуток времени:
    - тип `date` хранит только дату: год, месяц и день. Например, дату рождения или дату события.
    - тип `timestamp` хранит дату и время без учёта часового пояса.
    - тип `timestamptz`, или `timestamp with time zone`, хранит дату и время с учётом часового пояса. Система сохраняет время в UTC и конвертирует его в зависимости от настроек сессии. Этот тип подходит, чтобы записывать моменты событий.
    - тип `interval` хранит промежуток времени, например, "2 часа 30 минут". Его используют, чтобы проводить вычисления с датами.

5. **Логический тип `boolean`.** Этот тип хранит одно из трёх значений:
    - `TRUE` — истина;
    - `FALSE` — ложь;
    - `NULL` — неизвестное состояние.

Этот тип подходит для флагов и статусов. Например, поле `is_available` с типом `boolean` в таблице с автомобилей или поле `is_vip` в таблице клиентов.

## О поддержке неопределённых значений в реляционной базе данных

Мы разобрали основные типы данных в PostgreSQL. Теперь посмотрим, как система работает со значениями этих типов в SQL-запросах. Часто вам нужно явно указать, что строка `2024-01-30` — это дата, а не просто текст, или что число `1 — это логическое значение TRUE.

Когда вы пишете SQL-запрос, вы используете значения — константы. Система должна понять их тип. Для этого в SQL существуют два механизма: **явное указание типа** и **неявное преобразование**.

1. **Явное указание типа** — это механизм, который используют, когда нужно сказать СУБД, как интепретировать значение. Чтобы применить его, нужно использовать оператор `::` или функцию `CAST()`. 

    Например, для даты и времени `date`, `timestamp` и `timestamptz` строка `'2026-01-30'` сама по себе — это текст `text`. Чтобы СУБД поняла, что это дата, нужно применить оператор `::` или функцию `CAST()`.

    Допустим, нам нужно сравнить год выпуска автомобиля, который находится в колонке `year` типа `integer` с конкретной датой. Мы не можем сравнить число напрямую со строкой даты. Сначала нужно явно преобразовать строку в дату, а затем извлечь из неё год. Для этого мы можем использовать `CAST()`:

In [ ]:
%%sql
SELECT * FROM car 
WHERE year > EXTRACT(YEAR FROM CAST('2020-01-01' AS date));

Этот код сначала преобразует строку `'2020-01-01'` в дату с помощью `CAST('2020-01-01' AS date)`, затем функция `EXTRACT(YEAR FROM ... )` извлекает из этой даты числовой год, и система корректно сравнивает два целых числа: значение колонки `year` и число `2020`.

2. **Неявное преобразование** — это механизм, в ходе которого СУБД сама определяет тип значения на основе контекста. Система автоматически преобразует данные, если проводится логичная и безопасная операция.

    Например, если мы передаём целое число `1` в колонку с типом `numeric`, СУБД распознает это как `1.0`. Или если мы сравниваем целое число **2020** со строкой `2020` в условии `WHERE`, PostgreSQL часто сможет выполнить сравнение без ошибки.

    Давайте проверим это на простом примере, где система неявно преобразует целое число в текст для операции конкатенации:

In [ ]:
%%sql
SELECT 'Год выпуска: ' || year FROM car LIMIT 1;

В этом запросе оператор `||` соединяет строки. Колонка `year` имеет тип `integer`, но система автоматически, или **неявно**, преобразует числовое значение года в текст, чтобы выполнить конкатенацию со строкой `'Год выпуска: '`. СУБД сама определяет, что такое преобразование допустимо в данном контексте.

Мы научились работать с типами данных и их преобразованием. Теперь рассмотрим особую ситуацию, когда значение для атрибута может отсутствовать. Реляционная модель вводит специальное понятие для этого случая — значение `NULL`.

В реальных данных не все атрибуты обязательно заполнять. Например, у автомобиля в базе может быть не указан пробег, или у водителя может отсутствовать отчество. Для таких ситуаций в SQL существует специальное значение — `NULL`.

**Что означает NULL:**
- `NULL` означает отсутствие значения. Это маркер, который говорит СУБД, что значение для этого поля неизвестно, не применимо или просто не было указано.
- `NULL` не равно пустой строке `''`. Пустая строка — это валидное значение строкового типа, просто без символов. `NULL` — это отсутствие какого-либо значения вообще.
- `NULL` не равно нулю `0`. Ноль — это конкретное числовое значение. `NULL` — это отсутствие числа.
- `NULL` не равно пробелу `' '`. Пробел — это конкретный символ. `NULL` — это отсутствие данных.

Давайте проверим, как система различает `NULL` и пустую строку. Сначала посмотрим на данные, а затем попробуем найти строки, где значения нет.

In [ ]:
%%sql
SELECT cid, make, res_number FROM car WHERE res_number IS NULL;

Если бы этот запрос вернул строки, мы бы увидели реальный пример `NULL` в нашей базе данных, но во всех записях стоит какое-то число.

## Проверка наличия значения в поле

Мы узнали, что `NULL` означает отсутствие значения. Теперь важно понять, может ли поле вообще оставаться пустым. Когда вы проектируете таблицу, вы определяете для каждого поля правило: должно ли оно всегда содержать значение. От этого правила зависит, сможет ли система принять запись с пустым полем.

Ограничение `NOT NULL` указывает, что поле нужно обязательно заполнить. Если вы попытаетесь создать или обновить запись, не указав значение для такого поля, СУБД отклонит операцию с ошибкой.

Давайте проверим, как ограничение `NOT NULL` защищает данные на примере колонки `make`. В таблице `car` марка автомобиля — это критический атрибут. Без марки запись об автомобиле теряет смысл. Давайте попробуем нарушить это правило: попытаемся добавить автомобиль, но не будем указывать его марку.

In [ ]:
%%sql
INSERT INTO car (year, mileage, clsid) VALUES (2023, 0, 'A');

СУБД вернёт ошибку, `null value in column "make" violates not-null constraint`. СУБД проверила операцию `INSERT`, увидела, что для колонки `make`, которую объявили как `NOT NULL`, не передавали значение, и немедленно отклонила команду. Таким образом, ограничение `NOT NULL` физически не даёт появиться в таблице записям с пустым обязательным полем.

Мы выяснили, что система строго контролирует поля `NOT NULL` и возвращает ошибку, если видит нарушения. Но что происходит, когда `NULL` всё-таки попадает в вычисления или условия фильтрации? Работа с неопределёнными значениями требует особого подхода, потому что `NULL` ведёт себя не как обычное значение.

Язык SQL обрабатывает `NULL` по специальным логическим правилам. Эти правила влияют на результаты арифметических операций, конкатенации строк и, что самое важное, на отбор записей в условии `WHERE`.

**Всего таких правил три:**

1. **NULL «заражает» любую операцию.** Если в любой арифметической, строковой или логической операции участвует `NULL`, то результатом всей операции становится `NULL`. То есть неизвестное значение + какое-либо число = неизвестный результат.

    Давайте проверим, как это работает на простых выражениях:

In [ ]:
%%sql
SELECT 
    NULL + 5 AS результат_сложения,
    'Марка: ' || NULL AS результат_конкатенации,
    10 * NULL AS результат_умножения;

Все три колонки в результате будут содержать `NULL`. Этот подтверждает ключевое правило: `NULL` превращает в `NULL` результат любой операции, где он участвует. Это работает для сложения, вычитания, конкатенации строк и большинства других операторов.

2. **Чтобы проверить значение на `NULL`, нужны специальные операторы.** Обычные операторы сравнения: `=`, `<>`, `>`, `<` не работают с `NULL`. Сравнение значение = `NULL` всегда возвращает не `TRUE` и не `FALSE`, а `NULL`. Поэтому для проверки используют специальные операторы `IS NULL` и `IS NOT NULL`.

    Давайте проверим это на данных нашей таблицы `car`. Предположим, что мы хотим найти автомобили, для которых не указан номер брони `res_number`:

In [ ]:
%%sql
SELECT cid, make, res_number FROM car WHERE res_number = NULL;
SELECT cid, make, res_number FROM car WHERE res_number IS NULL;

Первый запрос с `res_number` = `NULL` вернёт пустой результат, даже если в таблице есть строки с `NULL` в этом поле. Это происходит потому, что для каждой строки сравнение `res_number` = `NULL` оценивается как `NULL`, а условие `WHERE` отбирает только строки, где выражение равно `TRUE`.

Второй запрос с `IS NULL` корректно находит все записи, где значения в колонке `res_number` нет.